<a href="https://colab.research.google.com/github/Malavika2609/text_summarization/blob/main/sentence_transformer/pdf_data_extraction_sentence_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from PyPDF2 import PdfReader

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load text from PDF (Assume you saved extracted text to "document.txt")





pdf_path = r"/content/ANTI FRAUD POLICY 05_11_2011.pdf"
reader = PdfReader(pdf_path)

# Extract text from all pages
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"

#print(text)

# Split text into sentences
sentences = text.split(". ")

# Convert sentences into embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
sentence_embeddings = embedding_model.encode(sentences)

# Create FAISS index for fast search
index = faiss.IndexFlatL2(sentence_embeddings.shape[1])
index.add(np.array(sentence_embeddings))

def find_relevant_context(question, top_k=3):
    """Finds the most relevant sentences from the document."""
    query_embedding = embedding_model.encode([question])
    _, indices = index.search(np.array(query_embedding), top_k)
    return " ".join([sentences[i] for i in indices[0]])

# Load pre-trained T5 model (you can choose t5-small, t5-base, t5-large, t5-3b, or t5-11b)
model_name = "t5-large"  # Change to "t5-base" or larger for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def summarize_text(text, max_length=150, min_length=50):
    question1="Definition of fraud and identification of fraud risk?"
    question2="Scope of the policy"
    question3="Requirements or procedures to prevent and detect fraud"
    question4="Requirements or procedures to investigate all reports or claims of fraud"
    question5="Requirements or procedures to tract incident or suspected fraud to resolution"
    question6="Requirements or procedures to report externally in any timeframes"


    context = find_relevant_context(question1)
    print('question:',question1)
    print('relevant answer:',context)
    summary1=find_summary_t5(question1,context)
    print('Summary:',summary1)

    context = find_relevant_context(question2)
    print('\nquestion:',question2)
    print('relevant answer:',context)
    summary2=find_summary_t5(question2,context)
    print('Summary:',summary2)

    context = find_relevant_context(question3)
    print('\nquestion:',question3)
    print('relevant answer:',context)
    summary3=find_summary_t5(question3,context)
    print('Summary:',summary3)

    context = find_relevant_context(question4)
    print('\nquestion:',question4)
    print('relevant answer:',context)
    summary4=find_summary_t5(question4,context)
    print('Summary:',summary4)

    context = find_relevant_context(question5)
    print('\nquestion:',question5)
    print('relevant answer:',context)
    summary5=find_summary_t5(question5,context)
    print('Summary:',summary5)

    context = find_relevant_context(question6)
    print('\nquestion:',question6)
    print('relevant answer:',context)
    summary6=find_summary_t5(question6,context)
    print('Summary:',summary6)


def find_summary_t5(question,context):
    input_text = f"question: {question}  context: {context}"
    #print("input text",input_text)
    #input_text = "Definition of fraud and identification of fraud risk " + text  # T5 requires task-specific prefixes
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    #print("input ",inputs)


    # Generate answer
    output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=512,  # 🔥 Increase max tokens
    min_length=50,  # 🔥 Force at least 50 tokens
    do_sample=True,
    temperature=0.9,
    top_p=0.95,
    top_k=50,
    repetition_penalty=1.2,
    length_penalty=2.5,  # 🔥 Force longer output
    no_repeat_ngram_size=3,
    num_return_sequences=1,
    early_stopping=False,  # 🔥 Prevent early stopping
    eos_token_id=None
    )


    # Generate summary
    output = model.generate(**inputs, max_new_tokens=50)
    #print("DEBUG: Raw Output:", output)

    #print(f"Generated Output Shape: {output.shape}")

    return tokenizer.decode(output[0], skip_special_tokens=True)




#text = """Artificial Intelligence (AI) is revolutionizing the world by enabling machines to mimic human intelligence, solve complex problems, and automate tasks that once required human intervention. AI encompasses various subfields, including machine learning, natural language processing, computer vision, and robotics, each contributing to advancements across industries. In healthcare, AI assists in diagnosing diseases, analyzing medical images, and developing personalized treatment plans, significantly improving patient outcomes.In finance, AI-driven algorithms detect fraudulent transactions, automate trading, and enhance risk assessment. The automotive industry is leveraging AI to develop self-driving cars that rely on deep learning and sensor fusion to navigate roads safely. AI-powered virtual assistants, such as Siri and Alexa, have transformed how we interact with technology, making information more accessible through voice commands. Moreover, AI plays a crucial role in business automation, optimizing supply chains, predicting market trends, and enhancing customer service through intelligent chatbots. Despite its numerous benefits, AI also raises ethical concerns, including biases in algorithms, job displacement due to automation, and the need for regulations to ensure responsible AI usage. As AI continues to evolve, researchers and policymakers must balance innovation with ethical considerations to create AI systems that are fair, transparent, and beneficial for society. The future of AI holds immense potential, from breakthroughs in scientific research to advancements in human-like reasoning, making it one of the most transformative technologies of the 21st century."""

summarize_text(text)
#print("Summary:", summary)

question: Definition of fraud and identification of fraud risk?
relevant answer: DEFINITION OF FRAUD  
Fraud is a false  representation or concealment of a material  fact or  any other illegal act 
committed intentionally to cause wrongful gains to self or others and /or wrongful loss to 
others ACTIONS CONSTITUTING FRAUD  
The terms fraud  or  suspected fraud refer s  to but not limited to: - 
(i) Forgery or alteration of any document  or account belonging to the 
Corporation                         CONTENTS  
I BACKGROUND  
II SCOPE OF POLICY  
III OBJECTIVES OF POLICY  
IV DEFINITION OF FRAUD  
V ACTIONS CONSTITUTING FRAUD  
VI OTHER INAPPR OPRIATE CONDUCT  
VII REPORTING PROCEDURE  
VIII (A) INVESTIGATION OF FRAUD/SUSPECTED FRAUD  
(B) TIME LIMITS FOR THE PRELIMINARY AND FINAL 
INVESTIGATION  
IX DISCIPLINARY ACTION  
X FRAUD PREVENTION  
XI MIS REPORT  
XII ADMINISTRATION OF THE POLICY  
 
 
 
 
 
2 
 ANTI FRAUD POLICY  
I
Summary: FRAUD

question: Scope of the policy
relevant ans

In [2]:
!pip install transformers sentence-transformers faiss-cpu
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin